# Desafío 6: Carlos Rubio

Para este desafío analizaremos las ventas de la empresa ADL con el fin de evaluar el desempeño de sus trabajadores en función de sus ventas.

In [2]:
#Cargamos la librería a utilizar
import pandas as pd

In [3]:
#Leemos el archivo con los datos de la empresa
excel_file = 'US_Regional_Sales_Data.xlsx'
xls = pd.ExcelFile(excel_file)

In [4]:
#Vemos que el archivo tiene varias pestañas, crearemos un DataFrame por cada pestaña.
sheet_names= pd.ExcelFile(excel_file).sheet_names
sheet_names

['Sales Orders Sheet',
 'Customers Sheet',
 'Store Locations Sheet',
 'Products Sheet',
 'Sales Team Sheet']

In [5]:
#Creamos un dataframe con todas las pestañas del archivo original
dataframes = {}
for sheet_name in sheet_names:
    dataframes[sheet_name] = pd.read_excel(excel_file, sheet_name=sheet_name)

In [6]:
#A cada item de nuestro dataframe, asignamos un dataframe respectivo a cada pestaña del archivo original
for sheet_name, df in dataframes.items():
    var_name = sheet_name.replace(' ', '_').lower() + '_df'
    globals()[var_name] = df

In [7]:
#Cruzaremos los dataframes por la columna respectiva, iremos sobreescribiendo la variable df_base
df_base = pd.merge(sales_orders_sheet_df,customers_sheet_df,on = '_CustomerID',validate= 'many_to_one')
df_base = pd.merge(df_base,store_locations_sheet_df, on = '_StoreID',validate='many_to_one')
df_base = pd.merge(df_base, products_sheet_df,on = '_ProductID',validate='many_to_one')
df_base = pd.merge(df_base, sales_team_sheet_df, on = '_SalesTeamID',validate='many_to_one')

In [8]:
#Agregamos las siguientes columnas a nuestro DataFrame
df_base['ProcurementDays'] = df_base['OrderDate']-df_base['ProcuredDate']
df_base['ShippingDays'] = df_base['ShipDate']-df_base['OrderDate']
df_base['DeliveryDays'] = df_base['DeliveryDate']-df_base['ShipDate']
df_base['CustomerDays'] = df_base['ShippingDays']+df_base['DeliveryDays']

In [9]:
#Clasificamos los valores de la columna 'CustomerDays' en los siguientes intervalos
Intervals=[0,15,30,45,60,75,90]
Labels = ['0 to 15 days','15 to 30 days','30 to 45 days','45 to 60 days','60 to 75 days','75 to 90 days']
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'].dt.days, bins=Intervals,labels=Labels)

In [10]:
#Generamos el reporte del conteo de órdenes gestionado por cada trabajador
df_Reporte_1 = pd.pivot_table(df_base,index='Sales Team',columns='CustomerDaysInterval',values='OrderNumber',aggfunc='count',fill_value=0,margins=True)
df_Reporte_1

CustomerDaysInterval,0 to 15 days,15 to 30 days,30 to 45 days,45 to 60 days,60 to 75 days,75 to 90 days,All
Sales Team,,,,,,,
Adam Hernandez,81,179,42,0,0,0,302
Anthony Berry,76,181,41,0,0,0,298
Anthony Torres,81,162,45,0,0,0,288
Carl Nguyen,96,178,40,0,0,0,314
Carlos Miller,65,148,34,0,0,0,247
Chris Armstrong,92,156,44,0,0,0,292
Donald Reynolds,97,163,36,0,0,0,296
Douglas Tucker,87,145,45,0,0,0,277
Frank Brown,85,150,40,0,0,0,275


In [11]:
#Agregamos la columna 'GrossMargin'
df_base['GrossMargin']=df_base['Order Quantity']*(df_base['Unit Price']*(1-df_base['Discount Applied'])-df_base['Unit Cost'])

In [12]:
#Clasificamos el margen bruto para asignar una comisión porcentual
Margins = [0,100,1000,10000,100000]
comission = [5,10,15,20]
df_base['ComissionPercentage'] = pd.cut(df_base['GrossMargin'],bins=Margins,labels=comission).astype(float) #Entrega valores tipo float

In [13]:
#Agregamos la columna 'CommissionsAmount'
df_base['CommissionsAmount'] = df_base['GrossMargin']*df_base['ComissionPercentage']/100

In [14]:
#Calculamos el margen neto 'NetMargin'
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

In [15]:
#Creamos una tabla pivote entre los vendedores y los márgenes obtenidos por cada uno.
df_Reporte_2 = pd.pivot_table(df_base,index='Sales Team',values=['GrossMargin','NetMargin','CommissionsAmount'],aggfunc='sum',margins=True).astype(float).round(2)
df_Reporte_2

,CommissionsAmount,GrossMargin,NetMargin
Sales Team,,,
Adam Hernandez,126637.21,779095.20,677987.33
Anthony Berry,124061.18,767943.95,654798.34
Anthony Torres,117962.21,743901.44,636758.05
Carl Nguyen,123843.21,774781.47,661874.73
Carlos Miller,110882.09,668752.93,580326.02
Chris Armstrong,122565.71,781608.70,660857.42
Donald Reynolds,143768.34,885386.81,754038.26
Douglas Tucker,120806.40,745528.56,633786.05
Frank Brown,104839.47,670881.25,570076.79
